In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

In [3]:
# dataset = "US Soybeans Futures Historical Data"
datasets = {
    "crude": "Crude Oil WTI Futures Historical Data",
    "feeder": "Feeder Cattle Futures Historical Data",
    "lean_hogs": "Lean Hogs Futures Historical Data",
    "cattle": "Live Cattle Futures Historical Data",
    "london_coffee": "London Robusta Coffee Futures Historical Data",
    "lumber": "Lumber Futures Historical Data",
    "oats": "Oats Futures Historical Data",
    "orange": "Orange Juice Futures Historical Data",
    "us_cocoa": "US Cocoa Futures Historical Data",
    "us_coffee_c": "US Coffee C Futures Historical Data",
    "soybean_oil": "US Soybean Oil Futures Historical Data",
    "soybeans": "US Soybeans Futures Historical Data",
    "sugar_11": "US Sugar #11 Futures Historical Data",
    "wheat": "US Wheat Futures Historical Data",
}

In [4]:
def normalize(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    df.reset_index(drop=True, inplace=True)

    # Convert price to float
    if df['Price'].dtype == 'O':
        df['Price'] = df['Price'].str.replace(',', '').astype(float)

    if df['Open'].dtype == 'O':
        df['Open'] = df['Open'].str.replace(',', '').astype(float)

    if df['High'].dtype == 'O':
        df['High'] = df['High'].str.replace(',', '').astype(float)

    if df['Low'].dtype == 'O':
        df['Low'] = df['Low'].str.replace(',', '').astype(float)

    # Convert Vol to float where K = 1000, M = 1000000
    if df['Vol.'].dtype == 'O':
        df['Vol.'] = df['Vol.'].str.replace('K', 'e3').str.replace('M', 'e6').astype(float)

    # Convert Change % to percentage float
    if df['Change %'].dtype == 'O':
        df['Change %'] = df['Change %'].str.replace('%', '').astype(float) / 100
        df['Change %'] = df['Change %'].round(4)

    return df

In [5]:
for k in datasets:
    print(f"Cleaning {k}")
    df = pd.read_csv(f"../raw/{datasets[k]}.csv")
    df = normalize(df)
    df.to_csv(f"../clean/{k}.csv", index=False)

Cleaning crude
Cleaning feeder
Cleaning lean_hogs
Cleaning cattle
Cleaning london_coffee
Cleaning lumber
Cleaning oats
Cleaning orange
Cleaning us_cocoa
Cleaning us_coffee_c
Cleaning soybean_oil
Cleaning soybeans
Cleaning sugar_11
Cleaning wheat


In [11]:
# Aggregate all data into one using "Date" and "Price" columns.
df = pd.DataFrame(columns=['Date'])
for k in datasets:
    print(f"Loading {k}")
    df1 = pd.read_csv(f"../clean/{k}.csv")
    df1 = df1[['Date', 'Price']]
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1.columns = ['Date', k]
    df = pd.merge(df, df1, on='Date', how='outer')

df = df.set_index('Date')
df = df.sort_index()

Loading crude
Loading feeder
Loading lean_hogs
Loading cattle
Loading london_coffee
Loading lumber
Loading oats
Loading orange
Loading us_cocoa
Loading us_coffee_c
Loading soybean_oil
Loading soybeans
Loading sugar_11
Loading wheat


In [12]:
# Fill NaN values with the previous value
df = df.fillna(method='ffill')
df

,crude,feeder,lean_hogs,cattle,london_coffee,lumber,oats,orange,us_cocoa,us_coffee_c,soybean_oil,soybeans,sugar_11,wheat
Date,,,,,,,,,,,,,,
2020-01-01,61.33,145.50,71.25,125.97,NaN,404.9,NaN,NaN,NaN,NaN,34.82,943.50,NaN,559.50
2020-01-02,60.95,144.65,71.55,125.78,1380.0,406.7,298.00,100.75,2532.0,NaN,35.24,944.25,13.13,560.25
2020-01-03,62.82,143.35,68.55,124.72,1372.0,403.6,290.75,99.40,2519.0,NaN,35.08,930.50,13.31,554.50
2020-01-06,63.04,147.43,68.62,127.28,1352.0,397.0,294.25,99.00,2484.0,NaN,34.52,932.75,13.73,550.00
2020-01-07,62.51,145.90,69.22,126.53,1364.0,395.4,293.25,99.65,2533.0,NaN,34.74,935.00,13.59,550.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,67.38,246.95,80.63,190.18,4502.0,535.5,379.75,530.95,6906.0,252.35,42.69,986.00,21.96,558.75
2024-10-29,67.21,244.03,82.82,189.30,4398.0,543.0,384.00,511.50,7264.5,248.10,42.80,979.00,22.08,570.50
2024-10-30,68.61,241.97,84.38,188.40,4453.0,553.5,390.75,505.35,7388.0,249.60,43.81,991.25,22.22,573.25


In [13]:
df.to_csv("../clean/all.csv")